In [1]:
import torch.nn as nn
import torch.nn.functional as F
import torch
import torchtext
import pandas as pd
import numpy as np
from torch.utils.data import TensorDataset, DataLoader, Dataset
from torch.nn.utils.rnn import pad_sequence
from sklearn.preprocessing import LabelEncoder
import re
import math

In [2]:
class network(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden_dim = 1
        self.n_layers = 1
        self.batch_size = 1
        
        self.gru = nn.GRU(10,self.hidden_dim,self.n_layers)#dummy numbers, figure it later
        self.fc = nn.Linear(self.hidden_dim,3)
        

    def forward(self, embedded):
        print(embedded)
        print(embedded.shape)
        hidden = self.init_hidden()
        out, hidden = self.gru(embedded, hidden)
        print(out)
        print(out.shape)
        out = self.fc(out)
        return out
    
    def init_hidden(self):
        weight = next(self.parameters()).data
        hidden = weight.new(self.n_layers, self.batch_size, self.hidden_dim).zero_().to(device)
        return hidden

In [3]:
def generate_batch(batch):#not sure if this is necessary anymore since im padding w/ different script
    batch_body = [t[0] for t in batch]
    batch_y = [t[1] for t in batch]

    lengths_x1 = torch.tensor([t.shape[0] for t in batch_body])
    b_x1 = pad_sequence(batch_body, batch_first=True, padding_value=0)
    batch_y = torch.tensor(batch_y)

    return b_x1, lengths_x1, batch_y


In [4]:
def train(model, dataLoader):
    acc = 0
    for i, data in enumerate(dataLoader):
        text = data[0]
        label = data[1]
        optimizer.zero_grad()
        output = model(text)
        loss = loss(output, label)
        loss.backward()
        optimizer.step()
        acc += (output.argmax(1) == label).sum().item()

    # Adjust the learning rate
    scheduler.step()
    acc = acc/len(dataLoader)
    print(f'Train Accuracy: {acc}')

In [13]:
def cleanLine(x):
    x = x[1:-1]
    x = re.findall(r"('.*?')", x)
    x = [i[1:-1] for i in x]
    return x

def splitLen(x):
    try:
        return len(x.split(' '))
        
    except:
        print(f"ERROR got: {x}")
        

class myDataset(Dataset):
    def __init__(self, csv, bodyNumpyList):
        df = pd.read_csv(csv)
        #self.body = pickle.load(bodyNumpyList)#this is a list of numpy arrays that have 0 padding at the beginning
        self.body = df['BodyCleaned']#.apply(lambda x: cleanLine(x))
        self.body.replace('', np.nan, inplace = True)
        self.body.dropna(inplace = True)
        maxLen =  df['BodyCleaned'].apply(splitLen).max()
        for idx, items in enumerate(df['BodyCleaned'].iteritems()): 
            if isinstance(items[1], float):
                print(idx, items[1])
        print(maxLen)
        uniqueList = []
        for line in self.body:
            for u in np.unique(line):
                uniqueList.append(u)
        uniqueList = np.array(uniqueList)
        self.label_encoder = LabelEncoder()
        self.label_encoder.fit_transform(np.unique(uniqueList))
        for i in self.body:
            print(i)
            print(type(i))
            break
        self.body = [np.concatenate(np.zeros(maxLen - len(i), 1), label_encoder.transform(i.split(' '))) for i in self.body]
        self.label = df['Y'].values.tolist()

        self.length = len(df)
       
    def __len__(self):
        return self.length
    
    def __getitem__(self,index):
        print(self.body[index])
        toReturn = [torch.tensor(self.body[index]), torch.tensor(np.array(self.label[index]))]
        return toReturn

In [14]:
EPOCH = 4
model = network()
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=4.0)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.9)
trainData = myDataset('train-cleaned.csv', 'bodyVectorized.pkl')#dummy value
trainDataLoader = DataLoader(trainData, batch_size=8, shuffle=True)
                  #collate_fn=generate_batch)
for _ in range(EPOCH):
    train(model, trainDataLoader)


19424


MemoryError: Unable to allocate 4.71 GiB for an array with shape (44996,) and data type <U28086

In [ ]:
#testData = TensorDataset(torch.from_numpy(trainX), torch.from_numpy(trainY))
testData  = myDataset('test-vectorized.csv', 'pklPath.pkl')#dummy values
testDataLoader = DataLoader(testData, batch_size=8, shuffle=True)
#                  collate_fn=generate_batch)
with torch.no_grad():
    acc = 0
    for data in testDataLoader:#[text, label]
        text = data[0]
        label = data[1]
        output = model(text)
        acc += (output.argmax(1) == label).sum().item()
    print(f'Testing accuracy: {acc/len(trainDataLoader)}')
    